# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [150]:
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

In [152]:
bank = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [154]:
bank.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [157]:
# Data Cleaning Notes

# features to drop completely: duration
bank = bank.drop('duration', axis=1)

# features to remove 'unknown' / 'nonexistent' values to make binary: default, housing, loan, poutcome
bank = bank[bank.default != 'unknown']
bank = bank[bank.housing != 'unknown']
bank = bank[bank.loan != 'unknown']
bank = bank[bank.poutcome != 'nonexistent']

bank['default'] = bank['default'].astype('category').cat.codes
bank['housing'] = bank['housing'].astype('category').cat.codes
bank['loan'] = bank['loan'].astype('category').cat.codes
bank['poutcome'] = bank['poutcome'].astype('category').cat.codes

# categorical features: job, marital, education, contact, month, day_of_week
bank['job'] = bank['job'].astype('category').cat.codes
bank['marital'] = bank['marital'].astype('category').cat.codes
bank['education'] = bank['education'].astype('category').cat.codes
bank['contact'] = bank['contact'].astype('category').cat.codes
bank['month'] = bank['month'].astype('category').cat.codes
bank['day_of_week'] = bank['day_of_week'].astype('category').cat.codes

# numeric features: age, campaign, pdays, previous, emp.var.rate, cons.price.idx, cons.conf.idx, euribor3m, nr.employed

# convert y column
bank['y'] = bank['y'].astype('category').cat.codes

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [160]:
bank.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4924 entries, 24013 to 41187
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             4924 non-null   int64  
 1   job             4924 non-null   int8   
 2   marital         4924 non-null   int8   
 3   education       4924 non-null   int8   
 4   default         4924 non-null   int8   
 5   housing         4924 non-null   int8   
 6   loan            4924 non-null   int8   
 7   contact         4924 non-null   int8   
 8   month           4924 non-null   int8   
 9   day_of_week     4924 non-null   int8   
 10  campaign        4924 non-null   int64  
 11  pdays           4924 non-null   int64  
 12  previous        4924 non-null   int64  
 13  poutcome        4924 non-null   int8   
 14  emp.var.rate    4924 non-null   float64
 15  cons.price.idx  4924 non-null   float64
 16  cons.conf.idx   4924 non-null   float64
 17  euribor3m       4924 non-null   f

### The business objective of the task is to extract from the data the most important features that lead to successful campaign results. The results of such analysis will help lead to more dedicated campaign efforts in the future in efforts of improving campaign results and reducing required resources for success.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [165]:
# Set X, y

In [167]:
X = bank.iloc[:,:-1] 
y = bank['y']

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [171]:
# Split test and train data

In [173]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                   random_state = 42)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [181]:
# Logistic Regression

# Initialize model
logreg = LogisticRegression(random_state=16)

# Fit the model and measure time
start_time = time.time()
logreg.fit(X_train, y_train)
logreg_fit_time = time.time() - start_time

# Make predictions
y_train_pred = logreg.predict(X_train)
y_test_pred = logreg.predict(X_test)

# Calculate train and test scores
logreg_train_score = mean_squared_error(y_train, y_train_pred)
logreg_test_score = mean_squared_error(y_test, y_test_pred)

print("Logistic Regression Train Score:", logreg_train_score)
print("Logistic Regression Test Score:", logreg_test_score)
print("Logistic Regression Fit Time:", logreg_fit_time)

Logistic Regression Train Score: 0.18792309775250474
Logistic Regression Test Score: 0.1803411860276198
Logistic Regression Fit Time: 0.09288191795349121


/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Problem 9: Score the Model

What is the accuracy of your model?

### The baseline model has a train accuracy of 0.18792309775250474 and a test accuracy of 0.1803411860276198.

### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [186]:
# Decision Tree
tree_depth_2 = DecisionTreeClassifier(max_depth=2).fit(X, y)

# Fit the model and measure time
start_time = time.time()
tree_depth_2.fit(X_train, y_train)
dectree2_fit_time = time.time() - start_time

# Make predictions
y_train_pred = tree_depth_2.predict(X_train)
y_test_pred = tree_depth_2.predict(X_test)

# Calculate train and test scores
dectree2_train_score = mean_squared_error(y_train, y_train_pred)
dectree2_test_score = mean_squared_error(y_test, y_test_pred)

print("Dec Tree 2 Train Score:", dectree2_train_score)
print("Dec Tree 2 Test Score:", dectree2_test_score)
print("Dec Tree 2 Fit Time:", dectree2_fit_time)

tree_depth_none = DecisionTreeClassifier(max_depth=None).fit(X, y)

# Fit the model and measure time
start_time = time.time()
tree_depth_none.fit(X_train, y_train)
dectree0_fit_time = time.time() - start_time

# Make predictions
y_train_pred = tree_depth_none.predict(X_train)
y_test_pred = tree_depth_none.predict(X_test)

# Calculate train and test scores
dectree0_train_score = mean_squared_error(y_train, y_train_pred)
dectree0_test_score = mean_squared_error(y_test, y_test_pred)

print("Dec Tree 0 Train Score:", dectree0_train_score)
print("Dec Tree 0 Test Score:", dectree0_test_score)
print("Dec Tree 0 Fit Time:", dectree0_fit_time)

Dec Tree 2 Train Score: 0.18981857568372598
Dec Tree 2 Test Score: 0.1803411860276198
Dec Tree 2 Fit Time: 0.005921840667724609
Dec Tree 0 Train Score: 0.0013539128080151638
Dec Tree 0 Test Score: 0.25832656376929325
Dec Tree 0 Fit Time: 0.017678022384643555


In [188]:
# KNN
knn_pipe_1 = Pipeline([('scale', StandardScaler()), 
                      ('model', KNeighborsClassifier(n_neighbors=1))]).fit(X, y)

# Fit the model and measure time
start_time = time.time()
knn_pipe_1.fit(X_train, y_train)
knn1_fit_time = time.time() - start_time

# Make predictions
y_train_pred = knn_pipe_1.predict(X_train)
y_test_pred = knn_pipe_1.predict(X_test)

# Calculate train and test scores
knn1_train_score = mean_squared_error(y_train, y_train_pred)
knn1_test_score = mean_squared_error(y_test, y_test_pred)

print("KNN 1 Train Score:", knn1_train_score)
print("KNN 1 Test Score:", knn1_test_score)
print("KNN 1 Fit Time:", knn1_fit_time)

knn_pipe_5 = Pipeline([('scale', StandardScaler()), 
                      ('model', KNeighborsClassifier(n_neighbors=5))]).fit(X, y)

# Fit the model and measure time
start_time = time.time()
knn_pipe_5.fit(X_train, y_train)
knn5_fit_time = time.time() - start_time

# Make predictions
y_train_pred = knn_pipe_5.predict(X_train)
y_test_pred = knn_pipe_5.predict(X_test)

# Calculate train and test scores
knn5_train_score = mean_squared_error(y_train, y_train_pred)
knn5_test_score = mean_squared_error(y_test, y_test_pred)

print("KNN 5 Train Score:", knn5_train_score)
print("KNN 5 Test Score:", knn5_test_score)
print("KNN 5 Fit Time:", knn5_fit_time)

KNN 1 Train Score: 0.0013539128080151638
KNN 1 Test Score: 0.255889520714866
KNN 1 Fit Time: 0.006979942321777344
KNN 5 Train Score: 0.15217979962090442
KNN 5 Test Score: 0.20471161657189277
KNN 5 Fit Time: 0.005349159240722656


In [190]:
# SVM
svc = SVC().fit(X_train, y_train)
svc_defaults = svc.score(X_test, y_test)

# Fit the model and measure time
start_time = time.time()
svc.fit(X_train, y_train)
svc_fit_time = time.time() - start_time

# Make predictions
y_train_pred = svc.predict(X_train)
y_test_pred = svc.predict(X_test)

# Calculate train and test scores
svc_train_score = mean_squared_error(y_train, y_train_pred)
svc_test_score = mean_squared_error(y_test, y_test_pred)

print("SVC Train Score:", svc_train_score)
print("SVC Test Score:", svc_test_score)
print("SVC Fit Time:", svc_fit_time)

SVC Train Score: 0.20417005144868672
SVC Test Score: 0.1917140536149472
SVC Fit Time: 0.1213831901550293


In [192]:
# Create lists for scores data frame
model_names = ["LogisticRegression", "DecisionTree", "KNN", "SVM"]
train_times = [logreg_fit_time, dectree2_fit_time, knn5_fit_time, svc_fit_time]
train_accuracies = [logreg_train_score, dectree2_train_score, knn5_train_score, svc_train_score]
test_accuracies = [logreg_test_score, dectree2_test_score, knn5_test_score, svc_test_score]

# Add to dataframe
model_scores = pd.DataFrame(list(zip(model_names, train_times, train_accuracies, test_accuracies)),
              columns=['Model','Train Time', 'Train Accuracy', 'Test Accuracy'])
model_scores.head(4)

,Model,Train Time,Train Accuracy,Test Accuracy
0,LogisticRegression,0.092882,0.187923,0.180341
1,DecisionTree,0.005922,0.189819,0.180341
2,KNN,0.005349,0.152180,0.204712
3,SVM,0.121383,0.204170,0.191714


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [201]:
# Try removing the age feature
bank_dropage = bank.drop('age', axis=1)

X = bank_dropage.iloc[:,:-1] 
y = bank_dropage['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                   random_state = 42)
# Initialize model
logreg_dropage = LogisticRegression(random_state=16)

# Fit the model and measure time
start_time = time.time()
logreg_dropage.fit(X_train, y_train)
logreg__dropage_fit_time = time.time() - start_time

# Make predictions
y_train_pred = logreg_dropage.predict(X_train)
y_test_pred = logreg_dropage.predict(X_test)

# Calculate train and test scores
logreg_dropage_train_score = mean_squared_error(y_train, y_train_pred)
logreg_dropage_test_score = mean_squared_error(y_test, y_test_pred)

# Try removing the job feature
bank_dropjob = bank.drop('job', axis=1)

X = bank_dropjob.iloc[:,:-1] 
y = bank_dropjob['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                   random_state = 42)
# Initialize model
logreg_dropjob = LogisticRegression(random_state=16)

# Fit the model and measure time
start_time = time.time()
logreg_dropjob.fit(X_train, y_train)
logreg__dropjob_fit_time = time.time() - start_time

# Make predictions
y_train_pred = logreg_dropjob.predict(X_train)
y_test_pred = logreg_dropjob.predict(X_test)

# Calculate train and test scores
logreg_dropjob_train_score = mean_squared_error(y_train, y_train_pred)
logreg_dropjob_test_score = mean_squared_error(y_test, y_test_pred)

# Try removing the marital feature
bank_dropmarital = bank.drop('marital', axis=1)

X = bank_dropmarital.iloc[:,:-1] 
y = bank_dropmarital['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                   random_state = 42)
# Initialize model
logreg_dropmarital = LogisticRegression(random_state=16)

# Fit the model and measure time
start_time = time.time()
logreg_dropmarital.fit(X_train, y_train)
logreg__dropmarital_fit_time = time.time() - start_time

# Make predictions
y_train_pred = logreg_dropmarital.predict(X_train)
y_test_pred = logreg_dropmarital.predict(X_test)

# Calculate train and test scores
logreg_dropmarital_train_score = mean_squared_error(y_train, y_train_pred)
logreg_dropmarital_test_score = mean_squared_error(y_test, y_test_pred)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite

In [203]:
# Create lists for scores data frame
linreg_model_names = ["All Features", "Drop Age", "Drop Job", "Drop Marital"]
linreg_train_times = [logreg_fit_time, logreg__dropage_fit_time, logreg__dropjob_fit_time, logreg__dropmarital_fit_time]
linreg_train_accuracies = [logreg_train_score, logreg_dropage_train_score, logreg_dropjob_train_score, logreg_dropmarital_train_score]
linreg_test_accuracies = [logreg_test_score, logreg_dropage_test_score, logreg_dropjob_train_score, logreg_dropmarital_train_score]

# Add to dataframe
linreg_scores = pd.DataFrame(list(zip(linreg_model_names, linreg_train_times, linreg_train_accuracies, linreg_test_accuracies)),
              columns=['Model','Train Time', 'Train Accuracy', 'Test Accuracy'])
linreg_scores.head(4)

,Model,Train Time,Train Accuracy,Test Accuracy
0,All Features,0.092882,0.187923,0.180341
1,Drop Age,0.069810,0.191172,0.178716
2,Drop Job,0.019918,0.190360,0.190360
3,Drop Marital,0.024519,0.187923,0.187923


### Based on the above comparison, it looks like dropping the job and marital features help for training the model. By dropping each of these features, we decrease the training time to fit the logistic regression model while either improving or keeping the same the train and test accuracies. However, we see that dropping the age feature while still improving training time to fit the logistic regression model, reduces test accuracy.

### From this comparison, we can gauge that the age feature is more important than the job and marital features. From this, we can determine that the age feature should be kept while the job and marital features are good options of features to drop from the data to improve the overall training time and performance of the model.

In [209]:
# Explore different number of neighbors for KNN

# KNN
knn_pipe_10 = Pipeline([('scale', StandardScaler()), 
                      ('model', KNeighborsClassifier(n_neighbors=10))]).fit(X, y)

# Fit the model and measure time
start_time = time.time()
knn_pipe_10.fit(X_train, y_train)
knn10_fit_time = time.time() - start_time

# Make predictions
y_train_pred = knn_pipe_10.predict(X_train)
y_test_pred = knn_pipe_10.predict(X_test)

# Calculate train and test scores
knn10_train_score = mean_squared_error(y_train, y_train_pred)
knn10_test_score = mean_squared_error(y_test, y_test_pred)

knn_pipe_15 = Pipeline([('scale', StandardScaler()), 
                      ('model', KNeighborsClassifier(n_neighbors=15))]).fit(X, y)

# Fit the model and measure time
start_time = time.time()
knn_pipe_15.fit(X_train, y_train)
knn15_fit_time = time.time() - start_time

# Make predictions
y_train_pred = knn_pipe_15.predict(X_train)
y_test_pred = knn_pipe_15.predict(X_test)

# Calculate train and test scores
knn15_train_score = mean_squared_error(y_train, y_train_pred)
knn15_test_score = mean_squared_error(y_test, y_test_pred)

In [211]:
# Create lists for scores data frame
knn_model_names = ["5 Nearest Neighbors", "10 Nearest Neighbors", "15 Nearest Neighbors"]
knn_train_times = [knn5_fit_time, knn10_fit_time, knn15_fit_time]
knn_train_accuracies = [knn5_train_score, knn10_train_score, knn15_train_score]
knn_test_accuracies = [knn5_test_score, knn10_test_score, knn15_test_score]

# Add to dataframe
knn_scores = pd.DataFrame(list(zip(knn_model_names, knn_train_times, knn_train_accuracies, knn_test_accuracies)),
              columns=['Model','Train Time', 'Train Accuracy', 'Test Accuracy'])
knn_scores.head(4)

,Model,Train Time,Train Accuracy,Test Accuracy
0,5 Nearest Neighbors,0.005349,0.152180,0.204712
1,10 Nearest Neighbors,0.004958,0.172759,0.189277
2,15 Nearest Neighbors,0.001366,0.178716,0.193339


### Based on the above comparison betwen 5, 10 and 15 nearest neighbors, we can see that the train times improve as we increase the number of neighbors. The train accuracies improve as we increase the number of neighbors as well. However, we see the best test accuracy is for only 5 neighbors. This leads to some interested tradeoff decisions. While we can increase number of neighbors to improve the train time and train accuracy, this needs to be weighed against the tradeoff of decrease in test accuracy.

### My recommendation here is to keep the number of neighbors at 5, as this provides the best test accuracy while only increasing the model fitting time by milliseconds.

In [215]:
# Exlore different max depth for Decision Tree
tree_depth_5 = DecisionTreeClassifier(max_depth=5).fit(X, y)

# Fit the model and measure time
start_time = time.time()
tree_depth_5.fit(X_train, y_train)
dectree5_fit_time = time.time() - start_time

# Make predictions
y_train_pred = tree_depth_5.predict(X_train)
y_test_pred = tree_depth_5.predict(X_test)

# Calculate train and test scores
dectree5_train_score = mean_squared_error(y_train, y_train_pred)
dectree5_test_score = mean_squared_error(y_test, y_test_pred)

tree_depth_10 = DecisionTreeClassifier(max_depth=10).fit(X, y)

# Fit the model and measure time
start_time = time.time()
tree_depth_10.fit(X_train, y_train)
dectree10_fit_time = time.time() - start_time

# Make predictions
y_train_pred = tree_depth_10.predict(X_train)
y_test_pred = tree_depth_10.predict(X_test)

# Calculate train and test scores
dectree10_train_score = mean_squared_error(y_train, y_train_pred)
dectree10_test_score = mean_squared_error(y_test, y_test_pred)

In [221]:
# Create lists for scores data frame
dectree_model_names = ["Depth 2", "Depth 5", "Depth 10"]
dectree_train_times = [dectree2_fit_time, dectree5_fit_time, dectree10_fit_time]
dectree_train_accuracies = [dectree2_train_score, dectree5_train_score, dectree10_train_score]
dectree_test_accuracies = [dectree2_test_score, dectree5_test_score, dectree10_test_score]

# Add to dataframe
dectree_scores = pd.DataFrame(list(zip(dectree_model_names, dectree_train_times, dectree_train_accuracies, dectree_test_accuracies)),
              columns=['Model','Train Time', 'Train Accuracy', 'Test Accuracy'])
dectree_scores.head(4)

,Model,Train Time,Train Accuracy,Test Accuracy
0,Depth 2,0.005922,0.189819,0.180341
1,Depth 5,0.009696,0.175196,0.190089
2,Depth 10,0.009632,0.108855,0.216897


### In comparing different depths for decision tree models, we have another tradeoff to consider. It appears test accuracy improves as depth increases. However, train time also increases and train accuracy decreases as we increase depth.

### My recommendation here would be to continue trying higher depth values. If higher depths continue to increase test accuracy, I recommend finding the depth where you are ok with the tradeoff in training time for the optimal test accuracy.

# Overall, based on the models analyzed in this notebook, I recommend the KNN model with 5 nearest neighbors as it has the best test accuracy of all the models tried. Also, it had the best train time compared to the other models.

##### Questions